In [1]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer, transpile
from qiskit.aqua import QuantumInstance
from qiskit import IBMQ
import math

In [2]:
def round(a,p,s,r,n,ctrl1,method='opt'):
    s_a=bin(int(a))[2:].zfill(n)
    s_p=bin(int(p))[2:].zfill(n)
    s_r=bin(int(r))[2:].zfill(n)
    s_s=bin(int(s))[2:].zfill(n)
    s_p=s_p[::-1]

    ctrl = QuantumRegister(1,'ctrl')
    u = QuantumRegister(n,'u')
    v = QuantumRegister(n,'v')
    ca = QuantumRegister(1,'ca')
    s = QuantumRegister(n,'s')
    r = QuantumRegister(n,'r')
    cb = QuantumRegister(1,'cb')
    cc = QuantumRegister(1,'cc')
    m = QuantumRegister(1,'m')
    clas_u = ClassicalRegister(n)
    clas_v = ClassicalRegister(n)
    clas_s = ClassicalRegister(n)
    clas_r = ClassicalRegister(n)
    circuit = QuantumCircuit(ctrl,u,v,ca,s,r,cb,cc,m,clas_u,clas_v,clas_s,clas_r)
    for i in range(0,n):
        if s_p[i]=='1':
            circuit.x(u[i])
        if s_a[i] == '1':
            circuit.x(v[n-1-i])
        if s_r[i] == '1':
            circuit.x(r[n-1-i])
        if s_s[i] == '1':
            circuit.x(s[n-1-i])
    if ctrl1=='1':
        circuit.x(ctrl)
    
    if method!='opt':
        sub_u = QuantumRegister(n)
        sub_v = QuantumRegister(n)
        sub_ca = QuantumRegister(1)
        sub_s = QuantumRegister(n)
        sub_r = QuantumRegister(n)
        sub_cb = QuantumRegister(1)
        sub_cc = QuantumRegister(1)
        sub_m = QuantumRegister(1)
        sub_circuit = QuantumCircuit(sub_u,sub_v,sub_ca,sub_s,sub_r,sub_cb,sub_cc,sub_m,name='block')
        ECC_library.block(sub_circuit,sub_u,sub_v,sub_ca,sub_s,sub_r,sub_cb,sub_cc,sub_m,n)
        sub_inst = sub_circuit.to_gate().control()
        circuit.append(sub_inst,ctrl[0:1]+u[0:n]+v[0:n]+ca[0:1]+s[0:n]+r[0:n]+cb[0:1]+cc[0:1]+m[0:1])
    else:
        ECC_library.controlled_block(circuit,ctrl,u,v,ca,s,r,cb,cc,m,n)

    for i in range(0,n):
        circuit.measure(u[i],clas_u[i])
    for i in range(0,n):
        circuit.measure(v[i],clas_v[i])
    for i in range(0,n):
        circuit.measure(s[i],clas_s[i])
    for i in range(0,n):
        circuit.measure(r[i],clas_r[i])
        
    return circuit

In [3]:
IBMQ.load_account();
provider = IBMQ.get_provider('ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')

v_v = [8]
v_u = [11]
v_r = [0]
v_s = [1]
t_vec = []
d_vec = []

a = 8 #v
p = 11 #u
s = 1
r = 0
n = 4
print('Optimized version')
while a>0:
    circuit = round(a,p,s,r,n,'1')
    d = circuit.decompose().decompose().decompose().depth()
    d_vec.append(d)
    simulation = execute(circuit, backend=backend ,shots=1)
    sim_result = simulation.result()
    T = sim_result.time_taken
    t_vec.append(T)
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_u = output[3]
    output_v = output[2]
    output_s = output[1]
    output_r = output[0]
    #output_ca = output[1]
    #output_m = output[0]
    res_u = int(output_u, 2)
    res_v = int(output_v, 2)
    res_s = int(output_s, 2)
    res_r = int(output_r, 2)
    #res_ca = int(output_ca, 2)
    #res_m = int(output_m, 2)
    print('res_u:',res_u,'res_v:',res_v,'res_s:',res_s,'res_r:',res_r,'      time:',T,'depth:',d)
    a=res_v
    p=res_u
    s=res_s
    r=res_r
    v_v.append(a)
    v_u.append(p)
    v_s.append(s)
    v_r.append(r)
print('--------------------------------')
print('Basic version')
v2_v = [8]
v2_u = [11]
v2_r = [0]
v2_s = [1]
t2_vec = []
d2_vec = []
a = 8 #v
p = 11 #u
s = 1
r = 0
n = 4
while a>0:
    circuit = round(a,p,s,r,n,'1',method='basic')
    d = circuit.decompose().decompose().decompose().depth()
    d2_vec.append(d)
    simulation = execute(circuit, backend=backend ,shots=1)
    sim_result = simulation.result()
    T = sim_result.time_taken
    t2_vec.append(T)
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_u = output[3]
    output_v = output[2]
    output_s = output[1]
    output_r = output[0]
    #output_ca = output[1]
    #output_m = output[0]
    res_u = int(output_u, 2)
    res_v = int(output_v, 2)
    res_s = int(output_s, 2)
    res_r = int(output_r, 2)
    #res_ca = int(output_ca, 2)
    #res_m = int(output_m, 2)
    print('res_u:',res_u,'res_v:',res_v,'res_s:',res_s,'res_r:',res_r,'      time:',T,'depth:',d)
    a=res_v
    p=res_u
    s=res_s
    r=res_r
    v2_v.append(a)
    v2_u.append(p)
    v2_s.append(s)
    v2_r.append(r)

Optimized version
res_u: 11 res_v: 4 res_s: 1 res_r: 0       time: 0.245550749 depth: 2122
res_u: 11 res_v: 2 res_s: 1 res_r: 0       time: 0.26985371 depth: 2122
res_u: 11 res_v: 1 res_s: 1 res_r: 0       time: 0.268530602 depth: 2122
res_u: 5 res_v: 1 res_s: 2 res_r: 1       time: 0.304086684 depth: 2122
res_u: 2 res_v: 1 res_s: 4 res_r: 3       time: 0.237710306 depth: 2122
res_u: 1 res_v: 1 res_s: 8 res_r: 3       time: 0.23396796700000003 depth: 2121
res_u: 1 res_v: 0 res_s: 11 res_r: 6       time: 0.23784653600000003 depth: 2122
--------------------------------
Basic version
res_u: 11 res_v: 4 res_s: 1 res_r: 0       time: 4.11906129 depth: 344040
res_u: 11 res_v: 2 res_s: 1 res_r: 0       time: 4.278347794 depth: 344040
res_u: 11 res_v: 1 res_s: 1 res_r: 0       time: 3.5423247750000004 depth: 344041
res_u: 5 res_v: 1 res_s: 2 res_r: 1       time: 4.277719897 depth: 344040
res_u: 2 res_v: 1 res_s: 4 res_r: 3       time: 3.672642476 depth: 344040
res_u: 1 res_v: 1 res_s: 8 res_r:

In [4]:
print('Optimized version')
print(v_u)
print(v_v)
print(v_r)
print(v_s)
print(t_vec)
print(d_vec)

print('--------------------------------')
print('Basic version')
print(v2_u)
print(v2_v)
print(v2_r)
print(v2_s)
print(t2_vec)
print(d2_vec)

Optimized version
[11, 11, 11, 11, 5, 2, 1, 1]
[8, 4, 2, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 1, 3, 3, 6]
[1, 1, 1, 1, 2, 4, 8, 11]
[0.245550749, 0.26985371, 0.268530602, 0.304086684, 0.237710306, 0.23396796700000003, 0.23784653600000003]
[2122, 2122, 2122, 2122, 2122, 2121, 2122]
--------------------------------
Basic version
[11, 11, 11, 11, 5, 2, 1, 1]
[8, 4, 2, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 1, 3, 3, 6]
[1, 1, 1, 1, 2, 4, 8, 11]
[4.11906129, 4.278347794, 3.5423247750000004, 4.277719897, 3.672642476, 4.33168854, 3.7965507830000003]
[344040, 344040, 344041, 344040, 344040, 344040, 344040]


In [5]:
print(sum(t_vec))

print(sum(t2_vec))

1.7975465540000002
28.018335555


In [8]:
for n in [4,5,6,7,8,9,10,11,12,13]:
    a=7
    p=11
    s=1
    r=0
    circuit = round(a,p,s,r,n,'1')
    d = circuit.decompose().decompose().decompose().decompose().depth()
    print('n:',n,'d:',d,'tot:',(d*2*n))

n: 4 d: 2122 tot: 16976
n: 5 d: 2805 tot: 28050
n: 6 d: 3508 tot: 42096
n: 7 d: 4231 tot: 59234
n: 8 d: 4954 tot: 79264
n: 9 d: 5673 tot: 102114
n: 10 d: 6472 tot: 129440
n: 11 d: 7169 tot: 157718
n: 12 d: 7848 tot: 188352
n: 13 d: 8531 tot: 221806
